# Compare OSM2GMNS Output to OSMnx and SharedStreets

In [9]:
import geopandas as gpd
import pandas as pd
import pyarrow.feather as arrow

from shapely import wkt
from geopandas.tools import sjoin

## Remote I/O

In [10]:
gmns_links_filename = "link.csv"
gmns_nodes_filename = "node.csv"
boundary_filename = "nw_pdx.geojson"
shst_filename = "shst.feather"
osmnx_filename = "osmnx_link_latest.geojson"

## Data Reads

In [11]:
osmnx_gdf = gpd.read_file(osmnx_filename)
link_df = pd.read_csv(gmns_links_filename)
node_df = pd.read_csv(gmns_nodes_filename)
nw_pdx_gdf = gpd.read_file(boundary_filename)
shst_df = arrow.read_feather(shst_filename)

In [12]:
join_node_df = node_df[["node_id", "osm_node_id"]]

In [13]:
df = link_df.copy()
df["geometry"] = df.rename(columns = {"geometry": "geo_string"}).geo_string.apply(wkt.loads)

link_gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
link_gdf = link_gdf[["link_id", "name", "osm_way_id", "from_node_id", "to_node_id", "geometry"]]

link_gdf = pd.merge(
    link_gdf,
    join_node_df,
    how="left",
    left_on="from_node_id",
    right_on="node_id",
).rename(columns={"osm_node_id":"u"}).drop("node_id", axis=1)

link_gdf = pd.merge(
    link_gdf,
    join_node_df,
    how="left",
    left_on="to_node_id",
    right_on="node_id",
).rename(columns={"osm_node_id":"v"}).drop("node_id", axis=1)

link_gdf = sjoin(link_gdf, nw_pdx_gdf, how="left")
link_gdf = link_gdf[link_gdf["id"].notnull()]
link_gdf = link_gdf.drop(["index_right", "id"], axis=1)

link_gdf.head()

,link_id,name,osm_way_id,from_node_id,to_node_id,geometry,u,v
26,26,Northwest Torreyview Drive,5386342,33,34,"LINESTRING (-122.76859 45.52197, -122.76890 45...",38704639.0,38704643.0
27,27,Northwest Torreyview Drive,5386342,34,33,"LINESTRING (-122.76905 45.52206, -122.76890 45...",38704643.0,38704639.0
32,32,Northwest 95th Avenue,5386696,39,40,"LINESTRING (-122.77542 45.52142, -122.77553 45...",38711211.0,38711213.0
33,33,Northwest 95th Avenue,5386696,40,39,"LINESTRING (-122.77602 45.52148, -122.77591 45...",38711213.0,38711211.0
34,34,NaN,5386821,44,45,"LINESTRING (-122.77548 45.52096, -122.77533 45...",38713215.0,38713221.0


In [14]:
osmnx_df = pd.DataFrame(osmnx_gdf[["u", "v", "osmid"]]).rename(columns={"osmid":"osmnx_osm_id"})
osmnx_df.head()

,u,v,osmnx_osm_id
0,40411137,40427408,5530927
1,40411137,4810850193,686713753
2,40411137,40479491,"[ 118255042, 837906539, 837906540 ]"
3,40411137,40562691,837906540
4,40566785,40526352,5525085


In [15]:
output_gdf = pd.merge(
    link_gdf,
    osmnx_df,
    how="left",
    on=["u","v"]
)

output_gdf = pd.merge(
    output_gdf,
    shst_df,
    how="left",
    on=["u","v"],
)

output_gdf.head()

,link_id,name,osm_way_id,from_node_id,to_node_id,geometry,u,v,osmnx_osm_id,shst_geometry_id
0,26,Northwest Torreyview Drive,5386342,33,34,"LINESTRING (-122.76859 45.52197, -122.76890 45...",38704639.0,38704643.0,5386342,13c4f787ffa1d64e57b29082ea92b017
1,27,Northwest Torreyview Drive,5386342,34,33,"LINESTRING (-122.76905 45.52206, -122.76890 45...",38704643.0,38704639.0,5386342,NaN
2,32,Northwest 95th Avenue,5386696,39,40,"LINESTRING (-122.77542 45.52142, -122.77553 45...",38711211.0,38711213.0,5386696,eab045284d4aaabe813dad9ba243841b
3,33,Northwest 95th Avenue,5386696,40,39,"LINESTRING (-122.77602 45.52148, -122.77591 45...",38711213.0,38711211.0,5386696,NaN
4,34,NaN,5386821,44,45,"LINESTRING (-122.77548 45.52096, -122.77533 45...",38713215.0,38713221.0,5386821,NaN


In [16]:
output_gdf.to_file("compare_osm2gmns.geojson", driver='GeoJSON')